# Aptamer Folding - 3dDNA & ViennaRNA
- Use ViennaRNA to 2d fold the target sequence
- Use 3dDNA to fold the sequence in tertiary space

- Software Requirement:
  - viennaRNA
  - 3dDNA


In [ ]:
import os, subprocess, requests, RNA

data_dir = "../../data"
os.chdir(data_dir)
print(os.getcwd())

## 1. ViennaRNA

In [ ]:
# specify the path to your FASTA file
fasta_file = "sequence.fasta"
output_fasta_file = "sequence_fold.fasta"

def read_fasta(fasta_file):
    """
    Parse the fasta file as user entered.

    Parameters:
        fasta_file (str): the sequence file directory
    Returns: 
        str: the sequence string
    """
    try:
        with open(fasta_file, 'r') as file:
            sequences = []
            sequence = ""
            for line in file:
                if line.startswith('>'):
                    if sequence:
                        sequences.append(sequence)
                    sequence = ""
                else:
                    sequence += line.strip()
            if sequence:
                sequences.append(sequence)
            if sequences:
                return sequences[0]  # Return the first sequence in the file
            else:
                raise ValueError("No sequences found in the FASTA file")
    except FileNotFoundError:
        raise FileNotFoundError("The specified FASTA file was not found")
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

def get_vienna_2d(sequence):
    """
    Use viennaRNA to get 2d conformation of the user input sequence

    Parameters:
        sequence (str): the user input sequence
    Returns: 
        str: the 2d conformation
    """
    fc = RNA.fold_compound(sequence)
    (ss, mfe) = fc.mfe()
    return ss

def write_to_fasta(output_fasta_file, vienna_2d):
    try:
        with open(output_fasta_file, 'w') as file:
            file.write(">Predicted_2D_Structure\n")
            file.write(sequence + "\n")
            file.write(vienna_2d + "\n")
    except Exception as e:
        raise Exception(f"An error occurred while writing to the output file: {e}")

try:
    sequence = read_fasta(fasta_file)
    vienna_2d = get_vienna_2d(sequence)
    print(f"Predicted 2D structure: {vienna_2d}")
    write_to_fasta(output_fasta_file, vienna_2d)
    print(f"The 2D structure has been written to {output_fasta_file}")
except Exception as e:
    print(e)


## 2. 3dDNA

- We use the ViennaRNA result for 3D-DNA server, which we will get several 3D folding conformations in PDB formats.
- The current API is not open yet, please contact the group directly or using the link below to access the server.
- Create your personal token or access server from http://biophy.hust.edu.cn/new/3dRNA/create


In [ ]:

auth_token = "[your-token-string]"
header = {'Authorization': 'Bearer ' + auth_token}

def send_request(data, headers):
    """
    Send a API request to the 3dDNA server

    Parameters:
        data (dict): A dictionary that works.
        headers (dict): authorization token for the 3dDNA server.
    Returns: 
        str: A status returned from the 3dDNA server.
    """
    try:
        response = requests.post('https://biophy.hust.edu.cn/api/3dRNA/jobs',
                                json=data, headers=headers, verify=False, timeout=60)
        response.raise_for_status()  # This will raise an HTTPError if the response returned an unsuccessful status code
        return response.json()["state"]
    except requests.exceptions.Timeout:
        return "The request timed out after 60 seconds"
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

data = {
    "name": "test",
    "params": {
        "seq": sequence,
        "ss": vienna_2d,
        "num": 5,
        "_minimize": "yes",
        "_pred_ss": "no",
        "_routine": "assemble"
    }
}

# submit a job
response_state = send_request(data, header)
print(response_state)


In [ ]:
# Get the result from this code
job_id = "Put_your_job_id_here"

try:
    response = requests.get(f"https://biophy.hust.edu.cn/api/3dRNA/jobs/{job_id}", headers=header, verify=False)
    response.raise_for_status()
    print(response.json())
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except requests.exceptions.RequestException as err:
    print(f"An error occurred: {err}")
except Exception as err:
    print(f"An unexpected error occurred: {err}")
